# Push Table Tags

This notebook provides an example for pushing table tags from Secoda to BigQuery. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [ ]:
%pip install google google-api-core google-api-python-client google-auth google-auth-httplib2 googleapis-common-protos httplib2

In [4]:

import requests
import httplib2
import google_auth_httplib2
from google.oauth2 import service_account
from googleapiclient.discovery import build

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"
BIGQUERY_PROJECT_ID = ""
BIGQUERY_JSON_CREDENTIALS = {}

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))

class BigQueryHelpers:
    DATE_LENGTH = 8
    DEFAULT_PAGE_SIZE = 300
    NUM_RETRIES = 1

    DEFAULT_SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

    @staticmethod
    def _is_sharded_table(table_id: str) -> bool:
        suffix = table_id[-BigQueryHelpers.DATE_LENGTH :]
        return suffix.isdigit()

http = httplib2.Http()
authed_http = google_auth_httplib2.AuthorizedHttp(
    # we have to re-create the credentials here instead of using the self.credentials
    # call due to the different scopes these credentials require
    service_account.Credentials.from_service_account_info(
        BIGQUERY_JSON_CREDENTIALS,
        scopes=BigQueryHelpers.DEFAULT_SCOPES,
    ),
    http=http,
)
bigquery_service = build(
    "bigquery", "v2", http=authed_http, cache_discovery=False
)


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Get Table Tags

You can obtain a table's tags with the table's ID or with the name, schema, and database. We'll use the name, schema, and database example

In [19]:
def get_tag(id):
    res = session.get(build_url(f"/tag/{id}/"))
    return res.json()

def get_tables(title, schema, database, *args, **kwargs):
    res = session.get(
        build_url(f"/table/tables/?integration=0fa11f8f-628c-4528-b195-75d08002f206")
    )
    results = res.json().get("results")
    if len(results) >= 1:
        return results
    return None

def patch_bigquery_table(table, properties):
    try:
        bigquery_service.tables().patch(
            projectId=BIGQUERY_PROJECT_ID,
            datasetId=table.get("properties", {}).get("_preview", {}).get("schema", ""),
            tableId=table.get("properties", {}).get("_preview", {}).get("title", ""),
            body=properties
        ).execute()
        print(f"Successfully updated table {table.get('title')} with {properties}")
    except:
        print(f"Failed to update table {table.get('title')} with {properties}")

tables = get_tables(**table)
for table in tables:
    
    # Push tags
    tags = table.get("tags", [])
    
    if table.get('pii', False):
        tags.append('pii')
        
    if table.get('verified', False):
        tags.append('verified')

    for tag_id in tags:
        if tag_id:
            
            if not tag_id in ['pii', 'verified']:
                tag = get_tag(tag_id)
            else:
                tag = {"name": tag_id}
        
        if not tag:
            continue
        
        patch_bigquery_table(table, {"labels": {tag.get("name").lower(): tag.get("name").lower()}})
        
    
    # Push descriptions
    if table.get('description', ''):
        patch_bigquery_table(table, {"description": table.get("description", "")})




{'name': 'verified'}
Successfully updated table customers with {'labels': {'verified': 'verified'}}
{'name': 'pii'}
Successfully updated table customers with {'labels': {'pii': 'pii'}}
{'name': 'verified'}
Successfully updated table customers with {'labels': {'verified': 'verified'}}
Successfully updated table stg_customers with {'description': 'test...test...'}
Successfully updated table ge_temp_228949d7 with {'description': 'test...test...'}
Successfully updated table my_second_dbt_model with {'description': 'test...test...'}
Successfully updated table payments with {'description': 'test...test...'}
